# MNIST Computer Vision Handwriting Recognition

I originally drafted this computer vision algorithm in MATLAB as part of an assignment for Andrew Ng's Machine Learning course on Coursera. To better practice industry tools in the Python scientific computing stack, I transcribed the code into Python (numpy, scipy) and reformatted it to be run in a Jupyter notebook. Note that this does not include a training/test split of the data.

In [33]:
import numpy as np
import math
import scipy.io as sio
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

Structure of the 3-layer neural network is as follows:

In [34]:
input_layer_size = 400 #20x20 input images of digits
hidden_layer_size = 25 #25 hidden units
output_layer_size = 10 #10 labels, from 1 to 10

In [35]:
def load_training_data(training_file='mnist_data.mat'):
    """
    Load training data. Return (inputs, labels).
    inputs - numpy array, size (5000,400).
    labels - numpy array, size (5000,1).
    
    This data can be found in Exercise 4 of the Coursera machine learning course and is named ex4data1.mat.
    """
    
    training_data = sio.loadmat(training_file)
    x = training_data['X']
    y = training_data['y']
    "training_data = sio.loadmat(training_file)"
    return (x,y)

In this exercise, the neural network parameters are pre-initialized, so we load those in as well. 

In [36]:
def load_weights(weight_file='initial_weights.mat'):
    weights = sio.loadmat(weight_file)
    theta1 = weights['Theta1']
    theta2 = weights['Theta2']
    return (theta1, theta2)

We continue by implementing the feedforward propagation that returns the cost only. 

In [37]:
def random_init_weights(input_size, output_size):
    epsilon_init = 0.12
    return np.random.rand(output_size, 1 + input_size) * 2 * epsilon_init - epsilon_init

In [38]:
def sigmoid(z):
    return 1 / (1 + pow(math.e, -z))

In [39]:
def gradient(z):
    return sigmoid(z) * (1 - sigmoid(z))

In [42]:
def nn_cost_function(theta1, theta2, input_layer_size, hidden_layer_size, output_layer_size, x, y, lambda_val=0):
    """
    Below are sizes of the numpy arrays passed as parameters into this function:
    theta1: (25, 401)
    theta2: (10, 26)
    x: (5000, 400)
    y: (5000, 1)
    
    """
    
    #preparing the layers of the neural net
    input_layer = np.insert(x, 0, 1, axis=1) #adding a bias column; input_layer dimensions (5000, 401)
    hidden_layer = np.dot(input_layer, np.transpose(theta1))
    hidden_layer = sigmoid(hidden_layer)
    hidden_layer = np.insert(hidden_layer, 0, 1, axis=1) #adding bias; hidden_layer dimensions (5000, 26)
    output_layer = np.dot(hidden_layer, np.transpose(theta2)) #5000x10
    output_layer = sigmoid(output_layer)
    
    #forward propagation
    cost = 0
    #transform each label in y from a number to a (10,1) vector -- binary notation
    #indexing starts from 1 and 0 is denoted as 10, e.g. if y[i] is 9, [0, 0, 0, 0, 0, 0, 0, 0, 1, 0]
    for index in range(len(x)):
        outputs = [0] * output_layer_size
        outputs[y[index]-1] = 1
    
    for k in range(output_layer_size):
        cost += -outputs[k] * math.log(output_layer[training_index][k]) - (1 - outputs[k]) * math.log(1 - output_layer[training_index][k])
        
    #back-propagation
    theta1_grad = np.zeros_like(theta1)  #25x401
    theta2_grad = np.zeros_like(theta2)  #10x26
    for i in range(len(x)):
        outputs = np.zeros((1, output_layer_size))  # (1,10)
        outputs[0][y[i]-1] = 1

        #calculate delta3
        delta3 = (output_layer[i] - outputs).T  # (10,1)

        #calculate delta2
        z2 = np.dot(theta1, input_layer[i:i+1].T)  # (25,401) x (401,1)
        z2 = np.insert(z2, 0, 1, axis=0)  # add bias, (26,1)
        #below: (26,10) x (10,1) leads to a (26,1) vector
        delta2 = np.multiply(np.dot(theta2.T, delta3), sigmoid_gradient(z2))
        delta2 = delta2[1:]  #(25,1)

        #(25,401) = (25,1) x (1,401)
        theta1_grad += np.dot(delta2, input_layer[i:i+1])
        #(10,26) = (10,1) x (1,26)
        theta2_grad += np.dot(delta3, hidden_layer[i:i+1])
    theta1_grad /= len(x)
    theta2_grad /= len(x)

    return cost, (theta1_grad, theta2_grad)

In [43]:
def gradient_descent(inputs, labels, learningrate=0.8, iteration=50):
    '''
    @return cost and trained model (weights).
    '''
    rand_theta1 = random_init_weights(input_layer_size, hidden_layer_size)
    rand_theta2 = random_init_weights(hidden_layer_size, output_layer_size)
    theta1 = rand_theta1
    theta2 = rand_theta2
    cost = 0.0
    for i in range(iteration):
        cost, (theta1_grad, theta2_grad) = nn_cost_function(theta1, theta2,
            input_layer_size, hidden_layer_size, output_layer_size,
            inputs, labels)
        theta1 -= learningrate * theta1_grad
        theta2 -= learningrate * theta2_grad
        print('Iteration {0} (learning rate {2}, iteration {3}), cost: {1}'.format(i+1, cost, learningrate, iteration))
    return cost, (theta1, theta2)


def train(inputs, labels, learningrate=0.8, iteration=50):
    cost, model = gradient_descent(inputs, labels, learningrate, iteration)
    return model


def predict(model, inputs):
    theta1, theta2 = model
    a1 = np.insert(inputs, 0, 1, axis=1)  # add bias, (5000,401)
    a2 = np.dot(a1, theta1.T)  # (5000,401) x (401,25)
    a2 = sigmoid(a2)
    a2 = np.insert(a2, 0, 1, axis=1)  # add bias, (5000,26)
    a3 = np.dot(a2, theta2.T)  # (5000,26) x (26,10)
    a3 = sigmoid(a3)  # (5000,10)
    return [i.argmax()+1 for i in a3]


if __name__ == '__main__':
    # Note: There are 10 units which present the digits [1-9, 0]
    # (in order) in the output layer.
    inputs, labels = load_training_data()

    model = train(inputs, labels, learningrate=0.1, iteration=60)

    outputs = predict(model, inputs)

    correct_prediction = 0
    for i, predict in enumerate(outputs):
        if predict == labels[i][0]:
            correct_prediction += 1
    precision = float(correct_prediction) / len(labels)
    print('precision: {}'.format(precision))


TypeError: only integer scalar arrays can be converted to a scalar index